In [1]:
import torch as th
import numpy as np
import pandas as pd
import os

In [ ]:
X.shape

In [ ]:
trans_clos.head()

In [ ]:
# mat = trans_clos.set_index('id').join(df )
# mat.head()

In [ ]:
trans_clos.shape

In [ ]:
df.shape

In [ ]:
df['id'] = df.id.astype(int)

In [ ]:

trans_clos['id'] = trans_clos.id.astype(int)

In [ ]:
mat_ = trans_clos.merge( df,on='id', how='left' )
mat_.head()

In [ ]:
mat_ = mat_.rename(columns={'id': 'id1', '1_x': 'id',   0: '1_x'})
mat_.head()

In [ ]:
mat_['id'] = mat_.id.astype(int)

In [ ]:
mat_ = mat_.merge( df,on='id', how='left' )
mat_.head()

In [ ]:
mat_ = mat_.rename(columns={'id': 'id2',   0: '2_x', 1: '2_y'})
mat_.head()

os.chdir('/home/ikira/poincare-embeddings/prepped_csv')
mat_.to_csv('alt_trans_clos_embeddings.csv', sep='\t', header=True, index= False)

In [2]:


os.chdir("../prepped_csv")
rel = pd.read_csv('top_levels.csv')

rel = rel.rename(columns={'weight': 'class label'})


rel.id1 = rel.id1.astype(int)
rel.id2 = rel.id2.astype(int)
rel.head()


id1        id2  class label
0  189723269  999999999            1
1  170589324  189723269            1
2  170589320  189723269            1
3  186201601  189723269            1
4  170589436  189723269            1

In [ ]:
rel[rel['id2'] == 189723269]

In [ ]:
#todo df with class labels, shares edge
done = mat_.merge( rel,on=['id1','id2'], how='left' )
done.head()

In [ ]:
done = done.fillna(0)
done.head()

In [ ]:
done.to_csv('alt_gold_trans_clos_embeddings.csv', sep='\t', header=True, index= False)

In [3]:
#NOW DO THE SAME WITH THE TRAINED EMBEDDINGS WITH DELETION AND STUFF

#load model 
os.chdir(home)
model = th.load("dim2_notrace_chrystall.pth.best")
model_val = model['model'] #ordered dict with additional infos
#weights are taken from tsv so far otherwise load from model[embeddings]
embeddings = model['embeddings'] #len 1180 type:tensor [1180, 5]
#either ID's as sreings but could also be the names of the objects
objects = model['objects'] # all ints


X_test = embeddings.numpy()


df_t = pd.DataFrame(data=X_test)
df_t['id'] = objects
df_t['id'] = df_t.id.astype(int)
# reoder columns
cols = df_t.columns.tolist()
cols = cols[-1:] + cols[:-1]
df_t = df_t[cols] 
df_t.head()

id         0         1
0  186201345 -0.950052 -0.013506
1  189721042 -0.886165 -0.013648
2  186131996 -0.317058 -0.627595
3  261481841 -0.299037 -0.619111
4  484995553  0.322711  0.400252

In [6]:
#load trans clos of that DS
os.chdir('../closure_csv')
trans_clos = pd.read_csv('top_lvl_no_trace_chrystallgrophy_ids.csv', header=None, usecols=[0,1], sep=",")
trans_clos = trans_clos[1:]

trans_clos = trans_clos.rename(columns={0: 'id', 1: 1})
trans_clos['id'] = trans_clos.id.astype(int)
trans_clos.head()

id          1
1  186201345  189721042
2  186131996  261481841
3  484995553  262491683
4  186201601  253227719
5  186201601  252912504

In [33]:
#SWAP COLUM AND COL NAMES THAT WAS IMPORTANT for the merge
#this shouldn't be too much of a problem because we look for undirected edges so a->b == b->a

swap = rel.reindex(columns=['id2','id1','class label'])
swap.head()
swap = swap.rename(columns={'id2':'id1', 'id1':'id2'})

In [8]:
mat_ = trans_clos.merge( df_t,on='id', how='left' )
mat_['id'] = mat_.id.astype(int)
mat_.head()

id        1_x         0       1_y
0  186201345  189721042 -0.950052 -0.013506
1  186131996  261481841 -0.317058 -0.627595
2  484995553  262491683  0.322711  0.400252
3  186201601  253227719 -0.598286 -0.800964
4  186201601  252912504 -0.598286 -0.800964

In [9]:
mat_ = mat_.rename(columns={'id': 'id1', '1_x': 'id',   0: '1_x'})
mat_.head()

id1         id       1_x       1_y
0  186201345  189721042 -0.950052 -0.013506
1  186131996  261481841 -0.317058 -0.627595
2  484995553  262491683  0.322711  0.400252
3  186201601  253227719 -0.598286 -0.800964
4  186201601  252912504 -0.598286 -0.800964

In [10]:
mat_['id'] = mat_.id.astype(int)


In [11]:
mat_ = mat_.merge( df_t,on='id', how='left' )

mat_ = mat_.rename(columns={'id': 'id2', 0: '2_x', 1:'2_y'})
mat_.head()

id1        id2       1_x       1_y       2_x       2_y
0  186201345  189721042 -0.950052 -0.013506 -0.886165 -0.013648
1  186131996  261481841 -0.317058 -0.627595 -0.299037 -0.619111
2  484995553  262491683  0.322711  0.400252  0.306095  0.413837
3  186201601  253227719 -0.598286 -0.800964 -0.425721 -0.806831
4  186201601  252912504 -0.598286 -0.800964  0.094071 -0.579875

In [34]:
#todo df with class labels, shares edge
done = mat_.merge(swap,on=['id1','id2'], how='outer')
done = done.fillna(0)
done.head()

id1        id2       1_x       1_y       2_x       2_y  class label
0  186201345  189721042 -0.950052 -0.013506 -0.886165 -0.013648          0.0
1  186131996  261481841 -0.317058 -0.627595 -0.299037 -0.619111          1.0
2  186131996  261481841 -0.317058 -0.627595 -0.299037 -0.619111          1.0
3  484995553  262491683  0.322711  0.400252  0.306095  0.413837          1.0
4  186201601  253227719 -0.598286 -0.800964 -0.425721 -0.806831          0.0

In [37]:
done[done['class label'] == 1.0]

id1        id2       1_x       1_y       2_x       2_y  \
1      186131996  261481841 -0.317058 -0.627595 -0.299037 -0.619111   
2      186131996  261481841 -0.317058 -0.627595 -0.299037 -0.619111   
3      484995553  262491683  0.322711  0.400252  0.306095  0.413837   
8      189720740  249563834  0.109796  0.922309  0.277798  0.401936   
10     189720915  261849767 -0.891375  0.345541 -0.889068  0.344052   
...          ...        ...       ...       ...       ...       ...   
15671  249672362  249672363  0.090182 -0.625291  0.110049 -0.629092   
15672  249672362  249672363  0.090182 -0.625291  0.110049 -0.629092   
15673  249672362  249672363  0.090182 -0.625291  0.110049 -0.629092   
15693  189967547  253045760  0.769439 -0.465798  0.760494 -0.460858   
15697  999999999  189723269  0.000000  0.000000  0.000000  0.000000   

       class label  
1              1.0  
2              1.0  
3              1.0  
8              1.0  
10             1.0  
...            ...  
15671          1.0  
15672          1.0  
15673          1.0  
15693          1.0  
15697          1.0  

[3849 rows x 7 columns]

In [68]:
sub = done[done['id1'] == 189723269 ]
ret = sub[sub['id2'] == 170589328]
ret
#social science and science have a class label of 0.
#where did I go wrong

id1        id2       1_x       1_y       2_x       2_y  \
6616  189723269  170589328 -0.042236 -0.002132 -0.527361  0.839004   

      class label  
6616          0.0

In [38]:
os.chdir('../prepped_csv')
done.to_csv('alt_test_chrystal_clos_embeddings.csv', sep='\t', header=True, index= False)